### Dask Architecture: Notebook (Client) + Scheduler (same node) + Workers (separate Slurm job)

Below is a conceptual diagram of the setup used in this session: a Jupyter notebook acts as the Dask client; the Dask scheduler runs on the same node as the notebook; and a separate Slurm job launches one or more worker processes on other nodes.

<div style="margin-top: 8px">
<svg width="900" height="360" viewBox="0 0 900 360" xmlns="http://www.w3.org/2000/svg" role="img" aria-label="Dask architecture diagram with client and scheduler on one node and workers in a separate Slurm job">
  <defs>
    <marker id="arrow" viewBox="0 0 10 10" refX="10" refY="5" markerWidth="7" markerHeight="7" orient="auto-start-reverse">
      <path d="M 0 0 L 10 5 L 0 10 z" fill="#3b82f6" />
    </marker>
    <style>
      .title { font: 700 16px sans-serif; fill: #111827; }
      .label { font: 600 13px sans-serif; fill: #111827; }
      .small { font: 12px sans-serif; fill: #374151; }
      .box { fill: #ffffff; stroke: #9ca3af; stroke-width: 1.5; rx: 8; }
      .box-blue { fill: #eff6ff; stroke: #3b82f6; }
      .box-green { fill: #ecfdf5; stroke: #10b981; }
      .cluster { fill: none; stroke: #9ca3af; stroke-dasharray: 6 6; rx: 10; }
      .link { stroke: #3b82f6; stroke-width: 2.2; marker-end: url(#arrow); }
      .link2 { stroke: #10b981; stroke-width: 2; marker-end: url(#arrow); }
    </style>
  </defs>

  <!-- Left cluster: Notebook + Scheduler on the same node -->
  <rect x="20" y="40" width="370" height="280" class="cluster" />
  <text x="205" y="30" text-anchor="middle" class="title">Node A (same job)</text>

  <!-- Notebook (Client) -->
  <rect x="45" y="80" width="160" height="70" class="box box-blue" />
  <text x="125" y="110" text-anchor="middle" class="label">Jupyter Notebook</text>
  <text x="125" y="128" text-anchor="middle" class="small">Dask Client</text>

  <!-- Scheduler -->
  <rect x="220" y="80" width="150" height="70" class="box box-blue" />
  <text x="295" y="110" text-anchor="middle" class="label">Dask Scheduler</text>
  <text x="295" y="128" text-anchor="middle" class="small">Ports: 8786/8787</text>

  <!-- Client <-> Scheduler link -->
  <line x1="205" y1="115" x2="220" y2="115" class="link" />

  <!-- Right cluster: Workers launched by a separate Slurm job -->
  <rect x="430" y="40" width="450" height="280" class="cluster" />
  <text x="655" y="30" text-anchor="middle" class="title">Worker nodes (separate Slurm job)</text>
  <text x="655" y="60" text-anchor="middle" class="small">srun/sbatch start dask-worker processes</text>

  <!-- Worker boxes -->
  <rect x="460" y="100" width="180" height="60" class="box box-green" />
  <text x="550" y="130" text-anchor="middle" class="label">Dask Worker</text>
  <text x="550" y="148" text-anchor="middle" class="small">nthreads x nprocs</text>

  <rect x="670" y="100" width="180" height="60" class="box box-green" />
  <text x="760" y="130" text-anchor="middle" class="label">Dask Worker</text>
  <text x="760" y="148" text-anchor="middle" class="small">GPU/CPU</text>

  <rect x="565" y="190" width="180" height="60" class="box box-green" />
  <text x="655" y="220" text-anchor="middle" class="label">(More Workers)</text>
  <text x="655" y="238" text-anchor="middle" class="small">scale up/down</text>

  <!-- Scheduler -> Workers links -->
  <path d="M 370 115 C 420 115, 420 130, 460 130" fill="none" class="link2" />
  <path d="M 370 115 C 450 95, 600 95, 670 130" fill="none" class="link2" />
  <path d="M 370 115 C 440 140, 520 180, 565 220" fill="none" class="link2" />

  <!-- Legend -->
  <rect x="35" y="335" width="12" height="12" class="box-blue" />
  <text x="55" y="345" class="small">Client/Scheduler</text>
  <rect x="180" y="335" width="12" height="12" class="box-green" />
  <text x="200" y="345" class="small">Workers</text>
  <line x1="300" y1="341" x2="340" y2="341" class="link" />
  <text x="348" y="345" class="small">control/status</text>
  <line x1="460" y1="341" x2="500" y2="341" class="link2" />
  <text x="508" y="345" class="small">task/data</text>
</svg>
</div>

Notes:
- The notebook and scheduler share the same node/job for low-latency control and dashboard access.
- Workers are launched by a separate Slurm allocation and register with the scheduler via its address.
- Scale the number of workers and threads per worker based on your workload and node resources.

In [ ]:
import numpy as np
import dask.array as da

In [ ]:
# AA = da.random.normal(0,1,size=(200000,200000), chunks=(1000, 1000))

In [ ]:
import socket
address = socket.gethostbyname(socket.gethostname())
print(address)

In [ ]:
from distributed import Client
client = Client(f'{address}:8786')

In [ ]:
client

Connect to the client using the "Dask Lab Extension" (red/orange icon on the far left of the JupyterLab screen) using `/proxy/8787/status` as the address.

<a href="/proxy/22222/status" target="_blank">Click here to access the Dask Dashboard directly using the Jupyter Server Proxy extension</a>

## Launch a dask distributed cluster

The easiest way is to use `dask-jobqueue`, see its documentation: https://jobqueue.dask.org/en/latest/configurations.html

However we are running inside a Singularity container which has no access to SLURM commands, therefore we manually launch the schedule in the Jupyterlab terminal with `launch_scheduler.sh` and submit a job for the workers using a terminal on the login node.

In [ ]:
AA = da.ones((200000,200000), chunks=(10000, 10000))

In [ ]:
AA.numblocks

In [ ]:
AA.nbytes / 1024**3

In [ ]:
%time AA.sum().compute()